## Load Libraries

In [ ]:
import os 
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.document_loaders import DirectoryLoader, CSVLoader

from dotenv import load_dotenv

## Load Model Groq from API

In [ ]:
load_dotenv(".env")
api_key = os.getenv("API_KEY")

model_llm = ChatGroq(groq_api_key=api_key,model_name="llama-3.1-8b-instant")
model_llm

## Load Embedding Model from  Hugging Face

In [ ]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name= embedding_model_name, model_kwargs=model_kwargs)

## Insert Data to VectorDB

In [ ]:
# Load CSV Files from Directory
folder_path = 'data/used_car_dataset.csv'
loader = CSVLoader(folder_path)
documents = loader.load_and_split()


print(f"Loaded {len(documents)} documents from CSV files.")

In [ ]:
vector_store = Chroma(
    collection_name="car_dataset",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
vector_store.add_documents(documents=documents)

## Prompt Template

In [ ]:
prompt_template = """
        You are an Helpful Assistant. The user will provide you with a CSV file and a question. You will answer the question based on the data in the CSV file.
        This is context from csv file:
        {context}
        This is the question:
        {question}
"""

In [ ]:
prompt = PromptTemplate.from_template(prompt_template)

## Chain

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
# Fungsi Generate Teks
def generate_teks(query:str, model = model_llm, retriever = retriever, prompt = prompt):
    qa_chain = RetrievalQA.from_chain_type(
                model,
                retriever=retriever,
                return_source_documents=False,
                chain_type_kwargs={"prompt": prompt}
                )
    response = qa_chain.invoke({"query": query})
    return response["result"]

## Inference

In [ ]:
generate_teks("Show me 3 Toyota Car")